# Generating data for Case studies

we will generate all the necessary data for the paper in this workbook

## Global positioning

In [ ]:
import numpy as np

#Define global Coordinates
globalPos = np.array([0,0,0])
accuraccy = 10

### Reference Data Selection

In [ ]:
from positioning import session

# Locating the Reference folder with all the reference sesisons
refSessionsPath = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data"
refSessions = session.find_close_sessions(refSessionsPath, globalPos, accuraccy)

## Local Alignment

In [ ]:
from positioning import session

# Getting the test session from the device/path
#testSessionPath = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-09-47"
testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 11-12-17"
testSession = session.Session().from_path(testSessionPath)
#test with 1 reference session
refSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42"
refSession = session.Session().from_path(refSessionPath)

### 2D Alignment

#### Showing all the matches

In [ ]:
from positioning.imagematch import ImageMatch

results = []

#test every test image against every reference image and get the match object to check the error and confidence
for refSession in refSessions:
    print("Matching reference session:", refSession.sessionId)

    for testImage in testSession.imageTransforms:
        print("Test Image:", testImage.id, "has a matchError of:")
        for refImage in refSession.imageTransforms:
            newMatch = ImageMatch(refImage, testImage)
            newMatch.find_matches() # find the best matches 
            results.append(newMatch)
            print(newMatch.matchError, "With image:", refImage.id)

#### Pose estimations method 1 (cross reference matching)
Using 2 separate matches to calculate the final pose

In [ ]:
import positioning.positioning2d as pos2d
from positioning.imagematch import ImageMatch

for testImage in testSession.imageTransforms:
    print("Test Image:", testImage.id, "matched with:")
    for refImage1 in refSession.imageTransforms:
        match1 = ImageMatch(refImage1, testImage)
        match1.find_matches()
        match1.get_essential_matrix()
        print("image1:", refImage1.id, "Error:", match1.matchError)
        for refImage2 in refSession.imageTransforms:
            if(refImage1.id == refImage2.id): continue # skip the same images
            match2 = ImageMatch(refImage2, testImage)
            match2.find_matches()
            match2.get_essential_matrix()
            print("image2:", refImage2.id, "Error:", match2.matchError)
            R,t, confidence = pos2d.cross_reference_pose(match1, match2)

            testOriginRot = testImage.get_rotation_matrix().T @ R
            testOrgininPos = - testOriginRot @ t
            testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
            print("Estimated pose wit confidence:", confidence)
            print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)



#### Pose estimation method 2 (incremental)
using 2 connected matches to find an estimation

In [ ]:
# we will test every test image against all the reference images and their best match

import positioning.positioning2d as pos2d
from positioning.imagematch import ImageMatch

for testImage in testSession.imageTransforms:
    print("Test Image:", testImage.id, "matched with:")
    for refImage in refSession.imageTransforms:
        testMatch = ImageMatch(testImage, refImage)
        refMatch = pos2d.get_best_session_match(refImage, refSession)

        R,t = testMatch.get_pnp_pose(refMatch) #get the rotation and translation with the pnp point algorithm
        confidence = testMatch.fidelity * refMatch.fidelity

        testOriginRot = testImage.get_rotation_matrix().T @ R
        testOrgininPos = - testOriginRot @ t
        testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
        print("Estimated pose wit confidence:", confidence)
        print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)

#### Pose estimation method 3 (raycasting)
using the 3D scene to get the session scale

In [ ]:
# we will test every test image against all the reference images and their best match

import positioning.positioning2d as pos2d
from positioning.imagematch import ImageMatch

for testImage in testSession.imageTransforms:
    print("Test Image:", testImage.id, "matched with:")
    for refImage in refSession.imageTransforms:
        match = ImageMatch(testImage, refImage)
        match.find_matches()
        match.get_essential_matrix()
        match.triangulate(useCameraPose = True)

        if(len(testSession.geometries) > 0):
            # the test sesison has geometry
            geometry = testSession.geometries[0]
            camera = testImage
        
        if(len(refSession.geometries) > 0):
            # the ref sesison has geometry
            geometry = refSession.geometries[0]
            camera = refImage

        geometry.get_distance_from_point(camera.pos, camera.rot)

        R,t = testMatch.get_pnp_pose(refMatch) #get the rotation and translation with the pnp point algorithm
        confidence = testMatch.fidelity * refMatch.fidelity

        testOriginRot = testImage.get_rotation_matrix().T @ R
        testOrgininPos = - testOriginRot @ t
        testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
        print("Estimated pose wit confidence:", confidence)
        print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)

## All the Estimations

In [ ]:
import positioning.positioning3d as pos3d
import open3d as o3d
# show all the estimations in 3d space from black to white, by confidence

points = []
colors = []

for estimation in testSession.estimations:
    points.append(estimation[1])
    colors.append(estimation[2] * [1,1,1])

estimatedPoints = o3d.geometry.PointCloud()
estimatedPoints.points = o3d.utility.Vector3dVector(points)
estimatedPoints.colors = o3d.utility.Vector3dVector(colors)

pos3d.show_geometries([estimatedPoints])

